In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import math

In [144]:
path = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/'
ffm_train = pd.read_csv(path + 'train_data.csv')
ffm_test = pd.read_csv(path + 'test_data.csv')
print (ffm_train.shape, ffm_test.shape)

(595212, 59) (892816, 58)


In [125]:
ffm_train_x = ffm_train.drop('target', axis=1)

list_cont = ['ps_car_13', 'ps_reg_03', 'ps_car_14', 'ps_car_12']
for i in list_cont:
    ffm_train[i] = pd.cut(ffm_train[i], 6, labels=False)
list_bin = []
for i in ffm_train_x.columns:
    if ffm_train_x[i].nunique() == 2:
        list_bin.append(i)
print (list_bin)

['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_car_08_cat', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']


In [126]:
for i in ffm_train.columns:
    if i not in list_bin and i not in ['id', 'target']:
#         print (i)
        encode = pd.get_dummies(ffm_train[i], prefix = i)
        ffm_train = ffm_train.join(encode).drop(i, axis=1)

In [51]:
ffm_train.to_csv(path + 'ffm_train_dataframe.csv', index=False)

In [130]:
ffm_test.shape

(892816, 492)

## Test_data

In [128]:
ffm_test['target'] = 0
ffm_test_x = ffm_test.drop('target', axis=1)

list_cont = ['ps_car_13', 'ps_reg_03', 'ps_car_14', 'ps_car_12']
for i in list_cont:
    ffm_test[i] = pd.cut(ffm_test[i], 6, labels=False)

In [129]:
for i in ffm_test.columns:
    if i not in list_bin and i not in ['id', 'target']:
#         print (i)
        encode = pd.get_dummies(ffm_test[i], prefix = i)
        ffm_test = ffm_test.join(encode).drop(i, axis=1)

In [118]:
# ffm_test['ps_calc_06_0'] = 0
# ffm_test['ps_calc_12_10'] = 0
# ffm_test['ps_car_12_4'] = 0

## LibSVM format

In [132]:
data = ffm_train
data1 = ffm_test

In [133]:
mapping = pd.read_csv(path + 'mapping_new.csv')

In [134]:
#field 
features = mapping['feature'].values 
mapping = mapping.sort_values(by=['field','index'])

In [135]:
index = mapping[['feature','index']].set_index('feature')[['index']].to_dict()['index'] 
field = mapping[['feature','field']].set_index('feature')[['field']].to_dict()['field']

In [142]:
# libsvm=[]
# for index1, row in data.iterrows():
    
#     string = ''
#     string += str(row['target']) + ' '
#     for feat in features:
#         value = int(float(row[feat])) 
#         if  value > 0:
#             string += str(field[feat]) + ':' + str(index[feat]) + ':' + str(value) + ' '
#     libsvm.append(string)
#     print (index1) 
# #     break
# print ('read')

In [139]:
data_libsvm = pd.DataFrame()
data_libsvm['libsvm_format'] = libsvm
print (data_libsvm.head(10)) 
data_libsvm.to_csv(path + 'data_libsvm_train_new.csv', index=False)

                                       libsvm_format
0  0 1:2:1 1:10:1 4:14:1 4:15:1 4:18:1 1:21:1 1:2...
1  0 1:3:1 1:11:1 3:12:1 4:14:1 4:15:1 4:17:1 1:2...
2  0 1:3:1 1:9:1 3:12:1 4:14:1 4:15:1 4:17:1 1:24...
3  0 1:1:1 1:9:1 3:12:1 1:19:1 1:28:1 1:34:1 1:45...
4  0 1:1:1 1:9:1 3:12:1 4:16:1 4:17:1 1:19:1 1:29...
5  0 1:4:1 1:9:1 3:12:1 4:14:1 4:16:1 4:17:1 4:18...
6  0 1:2:1 1:9:1 3:12:1 4:14:1 4:17:1 1:21:1 1:28...
7  0 1:1:1 1:9:1 3:12:1 4:13:1 4:15:1 4:17:1 1:24...
8  0 1:3:1 1:9:1 3:12:1 4:14:1 4:18:1 1:24:1 1:28...
9  1 1:2:1 1:11:1 3:12:1 4:14:1 4:17:1 1:20:1 1:2...


In [143]:
# libsvm=[]
# for index1, row in data1.iterrows():
    
#     string = ''
#     string += str(row['target']) + ' '
#     for feat in features:
#         value = int(float(row[feat])) 
#         if  value > 0:
#             string += str(field[feat]) + ':' + str(index[feat]) + ':' + str(value) + ' '
#     libsvm.append(string)
#     print (index1) 
# #     break
# print ('read')

In [141]:
data_libsvm = pd.DataFrame()
data_libsvm['libsvm_format'] = libsvm
print (data_libsvm.head(10)) 
data_libsvm.to_csv(path + 'data_libsvm_test_new.csv', index=False)

                                       libsvm_format
0  0 1:2:1 1:9:1 3:12:1 4:14:1 4:15:1 4:18:1 1:19...
1  0 1:4:1 1:9:1 3:12:1 4:15:1 4:16:1 4:18:1 1:23...
2  0 1:4:1 3:12:1 1:24:1 1:28:1 1:35:1 1:45:1 1:4...
3  0 1:1:1 1:9:1 3:12:1 4:13:1 4:15:1 1:19:1 1:28...
4  0 1:4:1 1:9:1 3:12:1 4:14:1 4:15:1 4:18:1 1:24...
5  0 1:1:1 1:9:1 4:13:1 4:15:1 4:17:1 1:19:1 1:28...
6  0 1:2:1 1:10:1 3:12:1 4:13:1 4:14:1 1:19:1 1:2...
7  0 1:1:1 1:9:1 3:12:1 4:14:1 1:19:1 1:28:1 1:32...
8  0 1:2:1 1:9:1 4:15:1 1:19:1 1:28:1 1:39:1 1:45...
9  0 1:4:1 1:10:1 3:12:1 4:14:1 4:15:1 1:20:1 1:2...


## Test_data submission

In [153]:
path_ffm = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/ffm/'
dt_final = pd.read_csv(path_ffm + 'out_file_new.csv', header=None)
dt_final.columns = ['target']

In [154]:
final_sub = ffm_test[['id']].join(dt_final['target'])
final_sub.to_csv(path_ffm + 'final_sub_ffm2.csv', index=False)

In [150]:
dt_final.head()

,target
0,0.500000
1,0.034300
2,0.039399
3,0.027810
4,0.019481


## Ensemble

In [3]:
path_new = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/porto/'
# xgb_submit = pd.read_csv(path_new + 'xgb_submit.csv')
# sub8 = pd.read_csv(path_new + 'final_sub8.csv')
# CNN = pd.read_csv(path_new + 'sub_286.csv')

CNN = pd.read_csv(path_new + 'NN_EntityEmbed_10fold-sub.csv')
xgb_submit = pd.read_csv(path_new + 'kagglemix.csv')
sub8 = pd.read_csv(path_new + 'sub_harmonic.csv')

CNN.columns = ['id','target_z']
xgb_submit.columns = ['id','target_x']
sub8.columns = ['id','target_y']

In [11]:
Ensemble_cnn_sub8.head()

,id,target_y,target_z,target_x,target
0,0,0.039599,0.063642,0.319118,0.098705
1,1,0.037633,0.050519,0.282084,0.085509
2,2,0.035949,0.048191,0.217847,0.074687
3,3,0.021835,0.031298,0.040522,0.030285
4,4,0.050036,0.069389,0.563168,0.146550


In [7]:
Ensemble_cnn_sub8 = CNN.merge(sub8, on='id', how='inner')
Ensemble_cnn_sub8['target'] = Ensemble_cnn_sub8.apply(lambda row: 1/(1+math.exp(-(math.log(row['target_z']/(1-row['target_z']))+math.log(row['target_y']/(1-row['target_y'])))/2)), axis=1)

In [10]:
Ensemble_cnn_sub8 = sub8.merge(CNN, on='id', how='inner').merge(xgb_submit, on='id', how='inner')
Ensemble_cnn_sub8['target'] = Ensemble_cnn_sub8.apply(lambda row: 1/(1+math.exp(-(math.log(row['target_x']/(1-row['target_x']))+math.log(row['target_z']/(1-row['target_z']))+math.log(row['target_y']/(1-row['target_y'])))/3)), axis=1)

In [12]:
Ensemble_cnn_sub8[['id', 'target']].to_csv(path_new + 'Ensemble_subharmonic_NN_Entity_kagglemix_logit.csv', index=False)